# Problem description

This notebook uses CP Optimizer to solve Sudoku problems.

# Reading the data

In [1]:
# This is the grid of the so-called "world's hardest sudoku" by Arto Inkala, 2012.
Grid = [
 [8,0,0,0,0,0,0,0,0],  
 [0,0,3,6,0,0,0,0,0], 
 [0,7,0,0,9,0,2,0,0], 
 [0,5,0,0,0,7,0,0,0], 
 [0,0,0,0,4,5,7,0,0], 
 [0,0,0,1,0,0,0,3,0], 
 [0,0,1,0,0,0,0,6,8], 
 [0,0,8,5,0,0,0,1,0],
 [0,9,0,0,0,0,4,0,0], 
]

# Modeling the problem with CP Optimizer

In [2]:
# Import Constraint Programming modelization functions
from docplex.cp.model import *

# Create model object
model = CpoModel()

# Decision variables
x = [ [integer_var(min=1,max=9) for c in range(9)] for r in range(9) ]


# Constraints: different values in each row
for r in range(9):
    model.add(all_diff([x[r][c] for c in range(9)]))


# Constraints: input grid values
for r in range(9):
    for c in range(9):
        if Grid[r][c]!=0:
            model.add(x[r][c] == Grid[r][c])

# Constraints: different values in each column    
for c in range(9):
    model.add(all_diff([x[r][c] for r in range(9)]))

# Constraints: different values in each sub-square 
for sr in range(3):
    for sc in range(3):
        model.add(all_diff([x[r][c] for r in range(3*sr,3*sr+3) for c in range(3*sc,3*sc+3)]))

# Solving the problem with CP Optimizer automatic search

The model can be solved by calling CP Optimizer's automatic search:

In [3]:
# Solve the model
sol = model.solve(trace_log=True)
#sol = model.solve(trace_log=True, Workers=1, AllDiffInferenceLevel='Extended')

 ! -------------------------------------------------- CP Optimizer 12.10.0.0 --
 ! Satisfiability problem - 81 variables, 48 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 122.9 (before), 122.9 (after)
 !  . Memory usage      : 335.2 kB (before), 335.2 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
                     1000         27    1         6 != _INT_30
                     1000         27    2         6 != _INT_30
                     1000         26    3   F     5 != _INT_73
                     1000         30    4   F     6  = _INT_47
                     1000         31    5   F     3  = _INT_5
                     1000         23    6         3  = _INT_28
                     1000         23    7   F     3  = _INT_9
                     1000         40    8         

# Displaying the input grid and the solution

In [4]:
print("Input grid:")
for r in range(9):
    print('                      ', end='')
    for c in range(9):
        if Grid[r][c]==0:
            s = '.'
        else:
            s = str(Grid[r][c])
            s = '\x1b[1;01;43m'+s  
        print(s+'\x1b[0m'+' ', end='')
    print()
print()

print("Solution:")
for r in range(9):
    print('                      ', end='')
    for c in range(9):
        s = str(sol.get_var_solution(x[r][c]).get_value())
        if Grid[r][c]!=0:
            s = '\x1b[1;01;43m'+s
        print(s+'\x1b[0m'+' ', end='')
    print()

Input grid:
                      8 . . . . . . . . 
                      . . 3 6 . . . . . 
                      . 7 . . 9 . 2 . . 
                      . 5 . . . 7 . . . 
                      . . . . 4 5 7 . . 
                      . . . 1 . . . 3 . 
                      . . 1 . . . . 6 8 
                      . . 8 5 . . . 1 . 
                      . 9 . . . . 4 . . 

Solution:
                      8 1 2 7 5 3 6 4 9 
                      9 4 3 6 8 2 1 7 5 
                      6 7 5 4 9 1 2 8 3 
                      1 5 4 2 3 7 8 9 6 
                      3 6 9 8 4 5 7 2 1 
                      2 8 7 1 6 9 5 3 4 
                      5 2 1 9 7 4 3 6 8 
                      4 3 8 5 2 6 9 1 7 
                      7 9 6 3 1 8 4 5 2 


In [5]:
# Import Constraint Programming modelization functions
from docplex.cp.model import *

# Create model object
model = CpoModel()

# Decision variables
x = [ [ integer_var(min=1,max=9) for j in range(9) ] for i in range(9) ]

# Objective function
model.add(minimize(sum([ x[i][i]+x[i][8-i] for i in range(9) ])))

# Constraints: different values in each row
for i in range(9):
    model.add(all_diff( [x[i][j] for j in range(9)] ))

# Constraints: different values in each column    
for j in range(9):
    model.add(all_diff( [x[i][j] for i in range(9)] ))

# Constraints: different values in each sub-square 
for si in range(3):
    for sj in range(3):
        model.add(all_diff( [x[i][j] for i in range(3*si,3*si+3) for j in range(3*sj,3*sj+3)] ))
        
sol = model.solve(trace_log=True, LogPeriod=1000000)

 ! -------------------------------------------------- CP Optimizer 12.10.0.0 --
 ! Minimization problem - 81 variables, 27 constraints
 ! LogPeriod            = 1000000
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 256.8 (before), 256.8 (after)
 !  . Memory usage      : 335.4 kB (before), 335.4 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         81                 -
 + New bound is 18
                        0         81    1            -
 + New bound is 19
                        0         81    1   F        -
 + New bound is 20
 *            92       54  0.04s        1      (gap is 78.26%)
 *            90      301  0.04s        1      (gap is 77.78%)
 *            82      941  0.04s        1      (gap is 75.61%)
 *            71      381  0.04s        3     

In [6]:
print("Solution:")
for r in range(9):
    print('                      ', end='')
    for c in range(9):
        s = str(sol.get_var_solution(x[r][c]).get_value())
        if r==c or r==8-c:
            s = '\x1b[1;01;43m'+s
        print(s+'\x1b[0m'+' ', end='')
    print()

Solution:
                      3 8 4 5 6 1 9 7 2 
                      9 1 7 4 2 8 5 3 6 
                      5 6 2 7 3 9 1 8 4 
                      6 2 8 3 9 4 7 1 5 
                      7 9 5 8 1 6 2 4 3 
                      1 4 3 2 7 5 8 6 9 
                      8 5 1 6 4 2 3 9 7 
                      4 3 9 1 5 7 6 2 8 
                      2 7 6 9 8 3 4 5 1 
